In [1]:
import numpy as np
import netCDF4 as nc
import os
import shutil
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
import pandas as pd
import pickle

%load_ext lab_black

- 需要注意读取的路径不能有中文的字符
- 目前的问题是用的是cmax，需要重新选择相应的公式  
- 目前已经成功获得了10%均匀采样的数据
- 注意pytho2和python3的区别，除法的结果
- tensor.view里面的-1表示infer的维度

In [ ]:
def plot_distribution(data_array: np.array):
    """
    输入二维的array，输出相应的面积图片
    这里人为定义了vmax的数值，之后可能需要改改
    """
    shape = data_array.shape
    x = np.arange(0, shape[1])  # len = 11
    y = np.arange(0, shape[0])  # len = 7

    fig, ax = plt.subplots(dpi=150)
    pcm = ax.pcolormesh(x, y, data_array, vmax=data_array.max(), vmin=0, cmap="Blues")
    fig.colorbar(pcm, ax=ax)


# plot_distribution(O3_con)

# fix the data size problem

In [28]:
base = "data/data-final-copy/"
for folder in os.listdir(base):
    names = os.listdir(base+folder)
    print(folder)
    for name in names:
        path = base + folder +"/" + name
        arr = np.load(path, allow_pickle=True)
        if arr.shape[1] == 34:
            arr = arr[:, 2:]
        np.save(path, arr)

O3_hourly_32_sh
O3_hourly_32_sh_sample_by_station
O3_hourly_32_sh_sample_by_station_test
O3_hourly_32_sh_test
PM25_hourly_32_sh
PM25_hourly_32_sh_sample_by_station
PM25_hourly_32_sh_sample_by_station_test
PM25_hourly_32_sh_test


In [29]:
base = "data/data-final-copy/"
for folder in os.listdir(base):
    names = os.listdir(base+folder)
    print(folder)
    for name in names:
        path = base + folder +"/" + name
        arr = np.load(path, allow_pickle=True)
        if arr.shape != (32,32):
            print("error on path: {}".format(path))
print("success")

O3_hourly_32_sh
O3_hourly_32_sh_sample_by_station
O3_hourly_32_sh_sample_by_station_test
O3_hourly_32_sh_test
PM25_hourly_32_sh
PM25_hourly_32_sh_sample_by_station
PM25_hourly_32_sh_sample_by_station_test
PM25_hourly_32_sh_test
success


# split the dataset to 90% training and 10% testing

In [ ]:
# finished
path = "data/data-final-copy/PM25_hourly_32_sh_sample_by_station"
os.mkdir(path+'_test')

names = pd.Series(os.listdir(path))
train_list = names.sample(frac=0.9, random_state=42)
test_list = set(names) - set(train_list)
print("the len of the test_list is {}".format(len(test_list)))
for name in test_list:
    # shutil.move(path+'/'+name, path+'_test'+'/'+name)
    print("moved "+name)


In [ ]:
# check the data
path = "data/data-final-copy/O3_hourly_32_sh_sample_by_station_test"
arr = np.load(path + "/" + os.listdir(path)[0], allow_pickle=True)
plot_distribution(arr)

In [ ]:
path = "data/data-final-copy/O3_hourly_32_sh"
arr = np.load(path + "/" + os.listdir(path)[0], allow_pickle=True)
plot_distribution(arr)

# change the image size to 32*32

In [ ]:
# check result
path = 'data/O3_hourly_32_sh/'
arr = np.load(path+os.listdir(path)[10], allow_pickle=True)
plot_distribution(arr)

In [ ]:
path = 'data/PM25_hourly_sample_by_station/'
arr = np.load(path+os.listdir(path)[0], allow_pickle=True)
plot_distribution(arr)

In [ ]:
base_folder = 'data/O3_hourly/'
save_folder = 'data/O3_hourly_32_sh/'
# os.mkdir(save_folder)
names = os.listdir(base_folder)
for i, name in enumerate(names[:5]):
    if (i%500 == 0): print('processed: {}/{}'.format(i, len(names)))
    arr = np.load(base_folder+name, allow_pickle=True)[65:97, 196:228]
    # np.save(save_folder+name, arr)
    plot_distribution(arr)

In [ ]:
base_folder = 'data/O3_hourly_sample_by_station/'
save_folder = 'data/O3_hourly_32_sh_sample_by_station/'
os.mkdir(save_folder)
names = os.listdir(base_folder)
for i, name in enumerate(names):
    if (i%500 == 0): print('processed: {}/{}'.format(i, len(names)))
    arr = np.load(base_folder+name, allow_pickle=True)[65:97, 194:228]
    np.save(save_folder+name, arr)

In [ ]:
base_folder = 'data/PM25_hourly/'
save_folder = 'data/PM25_hourly_32_sh/'
os.mkdir(save_folder)
names = os.listdir(base_folder)
for i, name in enumerate(names):
    if (i%500 == 0): print('processed: {}/{}'.format(i, len(names)))
    arr = np.load(base_folder+name, allow_pickle=True)[65:97, 194:228]
    np.save(save_folder+name, arr)

In [ ]:
base_folder = 'data/PM25_hourly_sample_by_station/'
save_folder = 'data/PM25_hourly_32_sh_sample_by_station/'
os.mkdir(save_folder)
names = os.listdir(base_folder)
for i, name in enumerate(names):
    if (i%500 == 0): print('processed: {}/{}'.format(i, len(names)))
    arr = np.load(base_folder+name, allow_pickle=True)[65:97, 194:228]
    np.save(save_folder+name, arr)

In [ ]:
plot_distribution(arr[80:112, 175:207])
arr[80:112, 175:207].shape

## 提取O3的数据

In [ ]:
def get_name_lis(basePath):
    """
    获得数据的名称列表
    """
    names = os.listdir(basePath)
    return names


basePath = "data/china_camx_original/"
names = get_name_lis(basePath)

In [ ]:
def process_netCDF_get_O3():
    """获得O3的数据，保存在data_O3_numpy里面"""
    basePath = "data/china_camx_original/"
    for name in names:
        path = basePath + name
        file_obj = nc.Dataset(path)
        O3 = file_obj.variables["O3"][:]
        O3_array = np.array([i[0] for i in O3])
        #     print(O3_array.shape)


#         np.save("data_O3_numpy/{}.npy".format(name[39:47]), O3_array)
# process_netCDF_get_O3()

# check data range

In [ ]:
base = "data/pm25_hourly/"
res = []
for name in os.listdir(base):
    path = base + name
    arr = np.load(path, allow_pickle=True)
    stat = [arr.min(), round(arr.max(), 4), round(arr.mean(), 4)]
    df = pd.DataFrame(arr.reshape((216 * 270, 1)))
    stat.append(df[df > 0].count() / (216 * 270.0))
    res.append(stat)
#     print(stat)

In [ ]:
# cal max and min
np.min([i[0] for i in res])

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot([i[0] for i in res])

In [ ]:
plt.figure(figsize=(50, 12), dpi=150)
plt.plot([i[1] for i in res])
plt.xlabel("sample number")
plt.ylabel("max O3 value")
plt.title("max O3 value in the whole china")
plt.savefig("saved/max_o3_hourly_sample.jpg")

In [ ]:
plt.figure(figsize=(50, 12), dpi=150)
plt.plot([i[2] for i in res])
plt.xlabel("sample number")
plt.ylabel("mean O3 value")
plt.title("mean O3 value in the whole china")
plt.savefig("saved/mean_o3_hourly_check.jpg")

# compare with original

In [ ]:
def cal_distance_helper(lon1, lat1, lon2, lat2):  # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    输出：距离km
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine公式
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r


def cal_distance(location1: tuple, location2: tuple) -> float:
    return cal_distance_helper(location1[0], location1[1], location2[0], location2[1])


def get_nearist_index(location_array: np.array, location: tuple) -> tuple:
    """
    input: the array stored all the location; the location
    output: the location index (i, j), not the (lon,lat)
    """
    minDis, index = None, (0, 0)
    for i in range(len(location_array)):
        for j in range(len(location_array[0])):
            dis = cal_distance(location, location_array[i][j])
            if not minDis:
                minDis = dis
            if dis < minDis:
                minDis = dis
                index = (i, j)
    return index


def get_location_array(basePath):
    # print(os.listdir())
    nameList = os.listdir(basePath)
    path = basePath + nameList[10]
    file_obj = nc.Dataset(path)
    lon = file_obj.variables["longitude"][:]
    lat = file_obj.variables["latitude"][:]

    res_arr = np.zeros((216, 270), dtype="object")
    for i in range(len(lon)):
        for j in range(len(lon[0])):
            res_arr[i][j] = (lon[i][j], lat[i][j])
    return res_arr

In [ ]:
def get_camx_station_data(loc: tuple, dataType="O3") -> pd.DataFrame:
    """
    input the station location (lon, lat), dataType = O3 or PM25
    return the O3 dateframe of that station
    """
    # loc -> index
    basePath = "data/china_camx_original/"
    arr = get_location_array(basePath)

    loc = get_nearist_index(arr, loc)

    base = "data/{}_hourly/".format(dataType)
    names = os.listdir(base)

    # get value from the index, add time info
    res = []
    for name in names:
        path = base + name
        arr = np.load(path, allow_pickle=True)
        res.append(
            [
                int(name[:4]),
                int(name[4:6]),
                int(name[6:8]),
                int(name[9 : name.find(".")]),
                arr[loc[0]][loc[1]],
            ]
        )
    # to dataframe and add timestemp
    df = pd.DataFrame(res, columns=["year", "month", "day", "hour", "value"])
    time_list = []
    for i, row in df.iterrows():

        string = (
            "-".join([str(int(i)) for i in [row.year, row.month, row.day]])
            + " "
            + str(int(row.hour))
            + ":00"
        )
        t = pd.Timestamp(string)
        time_list.append(t)
    df["pubtime"] = time_list
    return df


df = get_camx_station_data((87.5801, 43.8303), "O3")

In [ ]:
def get_monitor_data(path):
    """
    get the shanghai xuhui data
    """
    df_real = pd.read_csv(path)
    df_real["pubtime"] = pd.to_datetime(df_real.pubtime)
    return df_real


path = "data/xinjiang_wulumuqi_2019.csv"
df_real = get_monitor_data(path)

In [ ]:
df_m = pd.merge(df_real, df, on="pubtime")[["pubtime", "value", "o3"]]

df_m["value"] = df_m["value"].apply(lambda x: 1000 * 44.3 * x / 22.4)

df_m.index = pd.DatetimeIndex(df_m["pubtime"])
df_d = df_m.resample("d").mean()

In [ ]:
station = "shandong rizhao"
dataType = "O3"
plt.figure(figsize=(12, 6), dpi=200)
plt.plot(df_d)
plt.title("{} {} compare".format(station, dataType))
plt.xlabel("time")
plt.ylabel("{} value".format(dataType))
plt.savefig("saved/{}_{}_compare.jpg".format(station, dataType))

In [ ]:
plt.scatter(df_m.o3_value, df_m.o3, linewidths=0.001)

In [ ]:
plt.plot(df_merge.O3_value[:1000])

In [ ]:
plt.plot(df_merge.o3[:1000])

### 看看经度的左右是否符合要求

In [2]:
path = "data/china_camx_original/"
file_obj = nc.Dataset(path+os.listdir(path)[0])

In [3]:
lon = file_obj.variables["longitude"][:]
lat = file_obj.variables["latitude"][:]
round(lat.min(),2), round(lat.max(),2)

(15.03, 55.88)

In [7]:
lon[65:97, 196:228][0]

masked_array(data=[115.82800293, 116.03286743, 116.23760986, 116.44229126,
                   116.64685059, 116.85128784, 117.05566406, 117.2598877 ,
                   117.46405029, 117.66809082, 117.87200928, 118.07580566,
                   118.27954102, 118.48312378, 118.68658447, 118.88995361,
                   119.09320068, 119.29632568, 119.49932861, 119.70220947,
                   119.90499878, 120.1076355 , 120.31015015, 120.51254272,
                   120.71478271, 120.91693115, 121.118927  , 121.32080078,
                   121.52252197, 121.72412109, 121.92559814, 122.12692261],
             mask=False,
       fill_value=1e+20)

In [16]:
np.median([np.min(i) for i in lat[65:97, 196:228]]), np.median([np.max(i) for i in lat[65:97, 196:228]])

(30.627870559692383, 31.495628356933594)

In [18]:
_lat = lat[65:97, 196:228]
lat_new = [_lat[:, i] for i in range(32)]

In [20]:
np.median([np.min(i) for i in lat_new]), np.median([np.max(i) for i in lat_new])

(28.328094482421875, 33.88136672973633)

In [4]:
lon[65:97, 196:228].min(), lon[65:97, 196:228].max()

(115.8280029296875, 123.33212280273438)

In [39]:
lat[65:97, 196:228].min(), lat[65:97, 196:228].max()

(27.86715316772461, 34.28531265258789)

In [ ]:
file_obj.variables.keys()

In [ ]:
name_list = ['PNO3','PSO4','PNH4','POA','SOA1','SOA2',\
             'SOA3','SOA4','SOPA','SOPB','PEC','FPRM','FCRS']

In [ ]:
array = np.zeros((24, 1, 216, 270))
for name in name_list:
    print(name)
    array = array + np.array(file_obj.variables[name][:], dtype=float)

In [ ]:
plot_distribution(np.load("data_O3_sample_numpy/20190104-sample.npy")[0])

In [ ]:
array.max()

In [ ]:
file_obj.variables['PNO3'][:] + file_obj.variables['PSO4'][:]

In [ ]:
plot_distribution(np.load("data_O3_numpy/20190102.npy")[0])

## 将O3的数据进行采样，获得采样的数据

In [ ]:
names_o3 = get_name_lis("data_O3_numpy/")

In [ ]:
def get_sample_O3_data():
    '''将数据按照10%进行采样，得到相应的sample的数据，没采样的部分设置为0
    最后一天因为不够24hour，所以函数失败了'''
    xIndex = [i for i in range(0, 216, 10)]
    yIndex = [i for i in range(0, 270, 10)]

    for name in names_o3[:]:
        array_res = np.zeros((24, 216, 270))
        base = "data_O3_numpy/"
        array_24 = np.load(base+name)
        
    #     print(name)
        for hour in range(array_24.shape[0]):
            for x in xIndex:
                for y in yIndex:
                    array_res[hour][x][y] = array_24[hour][x][y]
    #         plot_distribution(array_res[hour])
        np.save("data_O3_sample_numpy/{}-sample.npy".format(name[:name.find(".")]), array_res)

In [ ]:
get_sample_O3_data()

In [ ]:
plot_distribution(np.load("data_O3_sample_numpy/20191226-sample.npy")[3])

## 尝试构建模型

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Generator(nn.Module):
    def __init__(self, nc, ngf):
        super(Generator,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(nc,ngf,kernel_size=4,stride=2,padding=1),
                                 nn.BatchNorm2d(ngf),
                                 nn.LeakyReLU(0.2,inplace=True))
        # 16 x 16 x 64
        self.layer2 = nn.Sequential(nn.Conv2d(ngf,ngf*2,kernel_size=4,stride=2,padding=1),
                                 nn.BatchNorm2d(ngf*2),
                                 nn.LeakyReLU(0.2,inplace=True))
        # 8 x 8 x 128
        
        self.layer3 = nn.Sequential(nn.Conv2d(ngf*2,ngf*4,kernel_size=4,stride=2,padding=1),
                                 nn.BatchNorm2d(ngf*4),
                                 nn.LeakyReLU(0.2,inplace=True))
        # 4 x 4 x 256                     
        # 4 x 4 x 256
        self.layer4 = nn.Sequential(nn.ConvTranspose2d(ngf*4,ngf*2,kernel_size=4,stride=2,padding=1),
                                 nn.BatchNorm2d(ngf*2),
                                 nn.ReLU())
        # 8 x 8 x 128
        self.layer5 = nn.Sequential(nn.ConvTranspose2d(ngf*2,ngf,kernel_size=4,stride=2,padding=1),
                                 nn.BatchNorm2d(ngf),
                                 nn.ReLU())
        # 16 x 16 x 64
        self.layer6 = nn.Sequential(nn.ConvTranspose2d(ngf,nc,kernel_size=4,stride=2,padding=1),
                                 nn.Tanh())
        # 32 x 32 x 1
    def forward(self,_cpLayer):
        out = self.layer1(_cpLayer)
        print(out.shape)
        out = self.layer2(out)
        print(out.shape)
        out = self.layer3(out)
        print(out.shape)
        out = self.layer4(out)
        print(out.shape)
        out = self.layer5(out)
        print(out.shape)
        out = self.layer6(out)
        print(out.shape)
        return out
class Discriminator(nn.Module):
    def __init__(self,nc,ndf):
        super(Discriminator,self).__init__()
        self.layer1_image = nn.Sequential(nn.Conv2d(nc,ndf//2,kernel_size=4,stride=2,padding=1),
                                 #nn.BatchNorm2d(ndf/2),
                                 nn.LeakyReLU(0.2,inplace=True))
        # 16 x 16
        self.layer1_cp = nn.Sequential(nn.Conv2d(nc,ndf//2,kernel_size=4,stride=2,padding=1),
                                 #nn.BatchNorm2d(ndf/2),
                                 nn.LeakyReLU(0.2,inplace=True))
        # 16 x 16
        self.layer2 = nn.Sequential(nn.Conv2d(ndf,ndf*2,kernel_size=4,stride=2,padding=1),
                                 nn.BatchNorm2d(ndf*2),
                                 nn.LeakyReLU(0.2,inplace=True))
        # 8 x 8
        
        self.layer3 = nn.Sequential(nn.Conv2d(ndf*2,ndf*4,kernel_size=4,stride=2,padding=1),
                                 nn.BatchNorm2d(ndf*4),
                                 nn.LeakyReLU(0.2,inplace=True))
        # 4 x 4
        
        self.layer4 = nn.Sequential(nn.Conv2d(ndf*4,1,kernel_size=4,stride=1,padding=0),
                                 nn.Sigmoid())
        # 1
#         self.layer5 = nn.Sequential(nn.Linear(, 1),
#                                  nn.Sigmoid())
        
    def forward(self,dem,_cpLayer):
        print(dem.shape)
        out_1 = self.layer1_image(dem)
        print(out_1.shape)
        out_2 = self.layer1_cp(_cpLayer)        
        out = self.layer2(torch.cat((out_1,out_2),1))
        print("concat", out.shape)
        out = self.layer3(out)
        print(out.shape)
        out = self.layer4(out)
        print(out.shape)
#         out = self.layer5(out)
#         print(out.shape)
        return out


In [ ]:
tmp = torch.tensor([[1,2,4],[2,5,3]])
tmp, tmp.view(1,-1)

In [ ]:
D = Discriminator(1, 32)

In [ ]:
# res = G.forward(img)
res = D.forward(img, img)
res.shape

In [ ]:
torch.randn(128, 20)[0].shape

In [ ]:
G = Generator(1, 64)
G = G.double()

In [ ]:
img.view(img.size()[0], -1)

In [ ]:
img

In [ ]:
arr = np.load("data_O3_sample_numpy/20191226-sample.npy")[3]

In [ ]:
arr[0]

In [ ]:
arr2 = arr[np.newaxis,np.newaxis,:,:]

In [ ]:
arr2.shape

In [ ]:
img = torch.from_numpy(arr2)

In [ ]:
img.double()

In [ ]:
plot = res.detach().numpy()

In [ ]:
res.shape

In [ ]:
plot_distribution(plot[0][0])